# SQL Final project

# Description

**A startup looking to develop a new online books platform, and want to conclude which publishers, authors and books are more popular by analyzing a database of one of the services competing in this market.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl

from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
def column_change(df, placed_column, replaced_column):
    temp = df[replaced_column].copy()
    df[replaced_column] = df[placed_column]
    df[placed_column] = temp
    df.rename(columns = {placed_column:'temp'}, inplace = True)
    df.rename(columns = {replaced_column:placed_column}, inplace = True)
    df.rename(columns = {'temp':replaced_column}, inplace = True)

In [4]:
def cell_change(df, column, placed_cell, replaced_cell):
    temp = df.loc[placed_cell, column]
    df.loc[placed_cell, column] = df.loc[replaced_cell, column] 
    df.loc[replaced_cell, column] = temp

In [5]:
pd.set_option('display.max_colwidth', 0)

In [6]:
dfs = {'books', 'authors', 'publishers', 'ratings', 'reviews'}

In [7]:
diagram = pd.DataFrame()
for df in sorted(dfs):
    query = ''' SELECT *
                FROM {}
                LIMIT 5
            '''.format(df)
    query = pd.io.sql.read_sql(query, con = engine)
    
    columns = pd.DataFrame(data = query.columns)
    columns.rename(columns = {0:df}, inplace = True)
    diagram = pd.concat([diagram, columns], axis = 1)

column_change(diagram, 'books', 'authors')
column_change(diagram, 'ratings', 'authors')
column_change(diagram, 'reviews', 'publishers')
cell_change(diagram, 'ratings', 1, 0)
cell_change(diagram, 'reviews', 1, 0)
cell_change(diagram, 'authors', 0, 1)
cell_change(diagram, 'publishers', 0, 5)
diagram = diagram.reindex([0,1,5,2,3,4])

diagram.style.applymap(lambda x: "color: red" 
                       if (x == 'book_id')
                       or (x == 'author_id')
                       or (x == 'author_id')
                       or (x == 'publisher_id')
                       else "color: black")

,books,ratings,reviews,authors,publishers
0,book_id,book_id,book_id,author,nan
1,author_id,rating_id,review_id,author_id,publisher
5,publisher_id,nan,nan,nan,publisher_id
2,title,username,username,nan,nan
3,num_pages,rating,text,nan,nan
4,publication_date,nan,nan,nan,nan


In [8]:
for df in dfs:
    query = ''' SELECT *
                FROM {}
                LIMIT 5
            '''.format(df)
    query = pd.io.sql.read_sql(query, con = engine)
    display(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**We have 4 dfs that connected to each other on the columns "book_id", "author_id", "publisher_id".**

# Task 1 - Find the number of books released after January 1, 2000

In [9]:
task_1 = ''' 
            WITH before_2000 AS (SELECT COUNT(book_id) as number_of_books_before_2000
                                 FROM books
                                 WHERE DATE_TRUNC('day', CAST(publication_date AS timestamp)) < '2000-01-01' ),
                 after_2000_01_01 AS (SELECT COUNT(book_id) as number_of_books_after_2000
                                      FROM books
                                      WHERE DATE_TRUNC('day', CAST(publication_date AS timestamp)) >= '2000-01-01' )             
            SELECT *
            FROM after_2000_01_01, before_2000
         '''
task_1 = pd.io.sql.read_sql(task_1, con = engine)
task_1

,number_of_books_after_2000,number_of_books_before_2000
0,821,179


**There are 821 books published after January 1, 2000 and only 179 books published before the year 2000. Looks like modern book or modern publishes are more popular and we should focus on them.**

# Task 2 - Find the number of user reviews and the average rating for each book.

In [14]:
task_2 = '''
            WITH num_of_reviews AS (SELECT book_id, COUNT(DISTINCT review_id) AS num_of_reviews
                                    FROM reviews
                                    GROUP BY book_id),
                                    
                  average_rating AS (SELECT book_id, AVG(rating) AS avg_rating
                                     FROM ratings
                                     GROUP BY book_id)
            SELECT b.book_id, b.title, re.num_of_reviews, ra.avg_rating
            FROM books AS b
            JOIN num_of_reviews AS re ON b.book_id = re.book_id
            JOIN average_rating AS ra ON b.book_id = ra.book_id
            ORDER BY num_of_reviews DESC, avg_rating DESC
            LIMIT 10
         '''
task_2 = pd.io.sql.read_sql(task_2, con = engine)
task_2

,book_id,title,num_of_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Olympians #1),6,4.080645
9,963,Water for Elephants,6,3.977273


**We need to find balance between the average score of a book, representing its quality, and the number of reviews that indicates popularity.**

# Task 3 - Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis)

In [11]:
task_3 = '''
            SELECT b.publisher_id, p.publisher, COUNT(b.book_id) AS num_of_books 
            FROM books AS b
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY num_of_books DESC
            LIMIT 10
         '''
task_3 = pd.io.sql.read_sql(task_3, con = engine)
task_3

,publisher_id,publisher,num_of_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19
5,33,Ballantine Books,19
6,45,Berkley,17
7,46,Berkley Books,14
8,284,St. Martin's Press,14
9,83,Delta,13


**Working with publishers that have higher number of books can help us have more variety on our platform.**

# Task 4 - Identify the author with the highest average book rating: look only at books with at least 50 ratings

In [12]:
task_4 = '''
            WITH over_50_ratings AS (SELECT book_id, AVG(rating) 
                                     FROM ratings
                                     GROUP BY book_id
                                     HAVING COUNT(rating_id) > 50)
                                     
            SELECT b.author_id, a.author, AVG(ra.avg) AS avg_book_rating
            FROM books AS b
            JOIN over_50_ratings AS ra ON b.book_id = ra.book_id
            JOIN authors AS a ON a.author_id = b.author_id
            GROUP BY b.author_id, a.author
            ORDER BY avg_book_rating DESC
         '''
task_4 = pd.io.sql.read_sql(task_4, con = engine)
task_4

,author_id,author,avg_book_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. Mowat,3.787879
9,106,Dan Brown,3.754540


**We can try focusing on authors that we know are popular, having high score ratings on books that have been rated at least 50 times.**

# Task 5 - Find the average number of text reviews among users who rated more than 50 books

In [13]:
task_5 = ''' 
            WITH rated_over_50 AS (SELECT username
                                   FROM ratings
                                   GROUP BY username
                                   HAVING COUNT(rating_id) > 50)
            SELECT AVG(count) AS average_text_reviews
            FROM (SELECT re.username, COUNT(re.review_id)
                  FROM reviews AS re
                  JOIN rated_over_50 AS ra ON ra.username = re.username
                  GROUP BY re.username) AS count
         '''
task_5 = pd.io.sql.read_sql(task_5, con = engine)
task_5

,average_text_reviews
0,24.333333


**Users that rate books often write a text review that we can analyze more easly.**

# Conclusion

**After analyzing the data, we can contact the bigger publishers and in their collection, find newer published books with higher ratings and higher number of reviews, by authors that usually perform good in average rating.**